In [372]:
import sys
import json

params = {'general':{'period':'am'}}
default = {'training_folder': '../..', 'params': params} # Default execution parameters
manual, argv = (True, default) if 'ipykernel' in sys.argv[0] else (False, dict(default, **json.loads(sys.argv[1])))
print(argv)


{'training_folder': '../..', 'params': {'general': {'period': 'am'}}}


In [373]:
import os
import time
import geopandas as gpd
import pandas as pd
sys.path.insert(0, r'../../../quetzal') # Add path to quetzal
from sklearn.neighbors import NearestNeighbors
from numba import jit, njit
import numba as nb
from quetzal.model import stepmodel
from shapely.geometry import LineString
from quetzal.io.gtfs_reader.importer import get_epsg
from quetzal.io import excel


In [374]:
period = argv['params']['general']['period']
on_lambda = bool(os.environ.get('AWS_EXECUTION_ENV'))
print('On Lambda : ', on_lambda)
training_folder = argv['training_folder']
input_folder = training_folder +r'/inputs/'

if on_lambda:
    scenario_folder = training_folder +r'/inputs/'
    output_folder = training_folder + '/outputs/'
    model_folder = training_folder + '/model/'
else:
    scenario_folder = training_folder + '/scenarios/' + period + '/inputs/'
    output_folder = training_folder + '/scenarios/' + period + '/outputs/'
    model_folder = training_folder + '/scenarios/' + period + '/model/'
print('input folder: ', input_folder)
print('output folder: ', output_folder)
print('scen folder : ', scenario_folder)
print('model folder : ', model_folder)


pt_folder = scenario_folder + 'pt/'
road_folder = input_folder+'base/' + 'road/'
if not os.path.exists(output_folder):
    os.makedirs(output_folder)

On Lambda :  False
input folder:  ../../inputs/
output folder:  ../../scenarios/am/outputs/
scen folder :  ../../scenarios/am/inputs/
model folder :  ../../scenarios/am/model/


In [375]:
var, ancestry = excel.read_var(file=input_folder+'/parameters.xlsx', scenario=period, return_ancestry=True)


In [376]:
var['zone_to_transit']['max_length']

10000000000.0

# inputs

In [377]:
links = gpd.read_file(pt_folder + 'links.geojson') 
nodes = gpd.read_file(pt_folder + 'nodes.geojson')
links = links.set_index('index')
nodes = nodes.set_index('index')

In [378]:
#links.index = 'link_' + links.index.astype(str)
#nodes.index = 'node_' + nodes.index.astype(str)
#links['a'] = 'node_' + links['a'].astype(str)
#links['b'] = 'node_' + links['b'].astype(str)

In [379]:
rnodes = gpd.read_file(road_folder + 'road_nodes.geojson')
rnodes = rnodes.set_index('index')
rlinks = gpd.read_file(road_folder + 'road_links.geojson')
rlinks = rlinks.set_index('index')

In [380]:
#rlinks.index = 'rlink_' + rlinks.index.astype(str)
#rnodes.index = 'rnode_' + rnodes.index.astype(str)
#rlinks['a'] = 'rnode_' + rlinks['a'].astype(str)
#rlinks['b'] = 'rnode_' + rlinks['b'].astype(str)

In [381]:
filepath = excel.get_filepath(input_folder + '{s}/zones/zones.geojson', ancestry=ancestry, log=False)
zones = gpd.read_file(filepath)
zones = zones.set_index('index')

# create Model

In [382]:
sm = stepmodel.StepModel(epsg=4326)
sm.links = links
sm.nodes = nodes
sm.road_links = rlinks
sm.road_nodes = rnodes
sm.zones = zones

In [383]:
centroid = [*LineString(sm.nodes.centroid.values).centroid.coords][0]
crs = get_epsg(centroid[1],centroid[0])
crs

32630

In [384]:
sm = sm.change_epsg(crs,'meter')

Reprojecting model from epsg 4326 to epsg 32630: 100%|█| 11/11 [00:03<00:00,  3.


# clustering and walkmodel

In [385]:
sm.preparation_clusterize_nodes(distance_threshold=200)


In [386]:
sm._add_type_prefixes({'nodes': 'node_','links':'links_','road_nodes':'rnode_','road_links':'rlink_','zones':'zone_'})

can not add prefixes on table:  footpaths


In [387]:
sm.road_links['a'] = 'rnode_' + sm.road_links['a'].astype(str)
sm.road_links['b'] = 'rnode_' + sm.road_links['b'].astype(str)

In [388]:
sm.preparation_footpaths(max_length = var['footpaths']['max_length'], speed = var['footpaths']['speed'])


In [389]:
# Zone to transit
sm.preparation_ntlegs(
    short_leg_speed = var['zone_to_transit']['speed'],
    long_leg_speed = var['zone_to_transit']['speed'], # tout le monde marche
    threshold = 1000,
    zone_to_transit = True,
    zone_to_road = False,
    road_to_transit = False,
    n_ntlegs = var['zone_to_transit']['nbr'],
    max_ntleg_length = var['zone_to_transit']['max_length'],
)
sm._add_type_prefixes({'zone_to_transit':'ztt_'})

In [390]:
sm.integrity_test_all(errors='ignore', verbose=True)

failed: integrity_test_circular_lines
passed: integrity_test_collision


14471it [00:00, 73201.87it/s]


passed: integrity_test_dead_ends
passed: integrity_test_duplicate_volumes
passed: integrity_test_isolated_roads
passed: integrity_test_nodeset_consistency
passed: integrity_test_road_duplicated_ab_links


14471it [00:00, 23796.50it/s]


passed: integrity_test_road_network
passed: integrity_test_road_nodeset_consistency
passed: integrity_test_sequences
passed: integrity_test_str_collision


In [391]:
only_attributes=['links', 'nodes', 'road_nodes', 'road_links', 'zones', 'coordinates_unit', 'epsg', 'footpaths',
                     'zone_to_transit', 'zone_to_road', 'road_to_transit', 'disaggregated_nodes']

if not os.path.exists(model_folder):
    os.makedirs(model_folder)

sm.to_zippedpickles(model_folder +'network', only_attributes=only_attributes)  

broken_sequences: 100%|█████████████████████████| 23/23 [00:02<00:00, 10.39it/s]
